In [1]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime
from strategy.ma_cross import MaCrossStrategy

In [2]:
# Create results directory if it doesn't exist
if not os.path.exists('backtest_results'):
    os.makedirs('backtest_results')

# Cerebro engine
cerebro = bt.Cerebro()
cerebro.addstrategy(MaCrossStrategy)

# Download AAPL_pd
AAPL_Data = yf.download('AAPL', start='2021-01-01', end='2023-01-01')

# Ensure correct column names
AAPL_Data = AAPL_Data.rename(columns={
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Adj Close': 'adj_close',
    'Volume': 'volume'
})

# Process column names before adding AAPL_pd
if hasattr(AAPL_Data.columns, 'levels'):  # If MultiIndex
    AAPL_Data.columns = ['_'.join(map(str, col)).strip() for col in AAPL_Data.columns.values]
else:
    AAPL_Data.columns = [str(col) for col in AAPL_Data.columns]

# Create AAPL_pd feed
AAPL_pd = bt.feeds.PandasData(
    dataname=AAPL_Data,
    datetime=None,  # If AAPL_Data index is datetime
    open=0,         # Adjust according to your column order
    high=1,
    low=2,
    close=3,
    volume=4,
    openinterest=-1
)

cerebro.adddata(AAPL_pd)

# Initial capital and commission
cerebro.broker.setcash(100000)
cerebro.broker.setcommission(commission=0.001)

# Run backtest
strategies = cerebro.run()
strat = strategies[0]  # Get your strategy object

# Save plot
plt.figure(figsize=(12, 8))
cerebro.plot(style='candle', volume=True, iplot=False)
plt.savefig('backtest_results/backtest_chart.png', dpi=300, bbox_inches='tight')
plt.close()

# Print important information
print("\n=== Backtest Results ===")
print(f"Final Portfolio Value: ${cerebro.broker.getvalue():.2f}")
print(f"Initial Capital: $100,000.00")
print(f"Net Profit/Loss: ${cerebro.broker.getvalue() - 100000:.2f}")
print(f"Return: {((cerebro.broker.getvalue() / 100000) - 1) * 100:.2f}%")
print(f"Total Trades: {strat.trade_count}")
print(f"Winning Trades: {strat.win_count}")
print(f"Losing Trades: {strat.loss_count}")
print(f"Total Profit/Loss: ${strat.total_profit:.2f}")
if strat.trade_count > 0:
    win_rate = strat.win_count / strat.trade_count * 100
    avg_profit_per_trade = strat.total_profit / strat.trade_count
    print(f"Win Rate: {win_rate:.2f}%")
    print(f"Average Profit per Trade: ${avg_profit_per_trade:.2f}")
else:
    print("Win Rate: No trades executed")

# Print each trade
if strat.trade_results:
    AAPL_Data_trades = pd.DataFrame(strat.trade_results)
    print("\n=== Trade History ===")
    print(AAPL_Data_trades)
    
    # Save to CSV for analysis
    AAPL_Data_trades.to_csv("backtest_results/trade_history.csv", index=False)
    print("\nTrade history saved to 'backtest_results/trade_history.csv'")
else:
    print("\n=== No Trade History ===")

# Save summary results to text file
with open('backtest_results/summary.txt', 'w') as f:
    f.write("=== Backtest Results Summary ===\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Symbol: AAPL\n")
    f.write(f"Period: 2021-01-01 to 2023-01-01\n")
    f.write(f"Strategy: Moving Average Crossover (20 periods)\n")
    f.write(f"Final Portfolio Value: ${cerebro.broker.getvalue():.2f}\n")
    f.write(f"Initial Capital: $100,000.00\n")
    f.write(f"Net Profit/Loss: ${cerebro.broker.getvalue() - 100000:.2f}\n")
    f.write(f"Return: {((cerebro.broker.getvalue() / 100000) - 1) * 100:.2f}%\n")
    f.write(f"Total Trades: {strat.trade_count}\n")
    f.write(f"Winning Trades: {strat.win_count}\n")
    f.write(f"Losing Trades: {strat.loss_count}\n")
    f.write(f"Total Profit/Loss: ${strat.total_profit:.2f}\n")
    if strat.trade_count > 0:
        win_rate = strat.win_count / strat.trade_count * 100
        avg_profit_per_trade = strat.total_profit / strat.trade_count
        f.write(f"Win Rate: {win_rate:.2f}%\n")
        f.write(f"Average Profit per Trade: ${avg_profit_per_trade:.2f}\n")

print("\nBacktest completed successfully!")
print("Results saved in 'backtest_results/' folder:")
print("- backtest_chart.png (chart plot)")
print("- trade_history.csv (detailed trades)")
print("- summary.txt (performance summary)")

C:\Users\Gary\AppData\Local\Temp\ipykernel_11460\437435.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  AAPL_Data = yf.download('AAPL', start='2021-01-01', end='2023-01-01')
[*********************100%***********************]  1 of 1 completed
c:\Users\Gary\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtrader\plot\plot.py:821: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  self.mpyplot.show()



=== Backtest Results ===
Final Portfolio Value: $99989.85
Initial Capital: $100,000.00
Net Profit/Loss: $-10.15
Return: -0.01%
Total Trades: 24
Winning Trades: 16
Losing Trades: 8
Total Profit/Loss: $3.24
Win Rate: 66.67%
Average Profit per Trade: $0.14

=== Trade History ===
   entry_time  entry_price  size  exit_time  exit_price     profit status
0  2021-02-02   131.533722     1 2021-02-16  129.973709   1.560013    win
1  2021-03-17   121.747314     1 2021-03-19  117.092484   4.654831    win
2  2021-03-24   117.190071     1 2021-03-26  118.283028  -1.092957   loss
3  2021-04-01   120.029808     1 2021-05-03  129.339401  -9.309593   loss
4  2021-06-08   123.889557     1 2021-08-02  142.247192 -18.357635   loss
5  2021-08-03   144.045792     1 2021-08-12  145.759476  -1.713684   loss
6  2021-08-13   145.965042     1 2021-08-20  145.074203   0.890839    win
7  2021-08-23   146.562225     1 2021-08-30  149.900513  -3.338287   loss
8  2021-08-31   148.637650     1 2021-09-14  145.005646 

<Figure size 1200x800 with 0 Axes>

In [ ]:
strat = strategies[0]  # 取出你的策略物件

# 印出重要資訊
print("\n=== 回測結果 ===")
print(f"期末資金: {cerebro.broker.getvalue():.2f}")
print(f"總交易次數: {strat.trade_count}")
print(f"獲利次數: {strat.win_count}")
print(f"虧損次數: {strat.loss_count}")
print(f"累積損益: {strat.total_profit:.2f}")
if strat.trade_count > 0:
    win_rate = strat.win_count / strat.trade_count * 100
    print(f"勝率: {win_rate:.2f}%")
else:
    print("勝率: 無交易紀錄")

# 印出每筆交易
if strat.trade_results:
    AAPL_Data_trades = pd.AAPL_Data(strat.trade_results)
    print("\n=== 交易紀錄 ===")
    print(AAPL_Data_trades)
    
    # 如果想存成 CSV 方便分析
    AAPL_Data_trades.to_csv("backtest_results.csv", index=False)
else:
    print("\n=== 無交易紀錄 ===")


=== 回測結果 ===
期末資金: 99989.85
總交易次數: 24
獲利次數: 16
虧損次數: 8
累積損益: 3.24
勝率: 66.67%

=== 交易紀錄 ===
   entry_time  entry_price  size  exit_time  exit_price     profit status
0  2021-02-02   131.533707     1 2021-02-16  129.973755   1.559952    win
1  2021-03-17   121.747299     1 2021-03-19  117.092461   4.654839    win
2  2021-03-24   117.190079     1 2021-03-26  118.283005  -1.092926   loss
3  2021-04-01   120.029793     1 2021-05-03  129.339401  -9.309608   loss
4  2021-06-08   123.889565     1 2021-08-02  142.247147 -18.357582   loss
5  2021-08-03   144.045776     1 2021-08-12  145.759460  -1.713684   loss
6  2021-08-13   145.965042     1 2021-08-20  145.074203   0.890839    win
7  2021-08-23   146.562210     1 2021-08-30  149.900543  -3.338333   loss
8  2021-08-31   148.637634     1 2021-09-14  145.005630   3.632004    win
9  2021-10-18   143.468658     1 2021-11-15  147.060440  -3.591782   loss
10 2021-11-16   148.040848     1 2022-01-07  168.795959 -20.755112   loss
11 2022-01-13   168.